In [54]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [112]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [113]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click',
       'user_gender_click', 'day_click'],
      dtype='object')

In [114]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum', 'user_click', 'shop_click', 'brand_click',
       'user_gender_click', 'item_click', 'day_click']

In [115]:
dataa = dataa[tezheng]

In [116]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [117]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 40)
2018-09-24 23:59:47
(357066, 40)
(121021, 40)


In [118]:

print('test')
test_a = pd.read_csv('test1.csv',index_col=False)
data2a = pd.read_csv('test1_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [119]:
data2a = data2a[tezheng]

In [120]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [121]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# del train['context_id']
# del val['context_id']
# del test_a['context_id']
# del train['item_brand_id']
# del val['item_brand_id']
# del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(42888, 35)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [122]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [123]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:11
2018-09-23 00:00:11
1 days 23:59:33
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(42888, 38)


In [124]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# del train['item_id']
# del test_a['item_id']
# del val['item_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_id_pvnum', 'occupation_click',
       'category_1_pvnum', 'user_click', 'shop_click', 'brand_click',
       'user_gender_click', 'item_click', 'day_click', 'category_0',
       'category_1', 'category_2', 'property_0', 'property_1', 'property_2',
       'halfhour', 'user_count', 'item_count'],
      dtype='object')
(520975, 595)


In [125]:
# LogisticRegression?

In [126]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [127]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [128]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683903
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674842
[3]	valid_0's binary_logloss: 0.665957
[4]	valid_0's binary_logloss: 0.657246
[5]	valid_0's binary_logloss: 0.648695
[6]	valid_0's binary_logloss: 0.640306
[7]	valid_0's binary_logloss: 0.632077
[8]	valid_0's binary_logloss: 0.624013
[9]	valid_0's binary_logloss: 0.616094
[10]	valid_0's binary_logloss: 0.608314
[11]	valid_0's binary_logloss: 0.600683
[12]	valid_0's binary_logloss: 0.593191
[13]	valid_0's binary_logloss: 0.585838
[14]	valid_0's binary_logloss: 0.57862
[15]	valid_0's binary_logloss: 0.571521
[16]	valid_0's binary_logloss: 0.564548
[17]	valid_0's binary_logloss: 0.557692
[18]	valid_0's binary_logloss: 0.550958
[19]	valid_0's binary_logloss: 0.544349
[20]	valid_0's binary_logloss: 0.537852
[21]	valid_0's binary_logloss: 0.531473
[22]	valid_0's binary_logloss: 0.525188
[23]	valid_0's binary_logloss: 0.519014
[24]	valid_0's binary_loglo

[203]	valid_0's binary_logloss: 0.125545
[204]	valid_0's binary_logloss: 0.124993
[205]	valid_0's binary_logloss: 0.124449
[206]	valid_0's binary_logloss: 0.12391
[207]	valid_0's binary_logloss: 0.123376
[208]	valid_0's binary_logloss: 0.122852
[209]	valid_0's binary_logloss: 0.122334
[210]	valid_0's binary_logloss: 0.121821
[211]	valid_0's binary_logloss: 0.121314
[212]	valid_0's binary_logloss: 0.120814
[213]	valid_0's binary_logloss: 0.12032
[214]	valid_0's binary_logloss: 0.119832
[215]	valid_0's binary_logloss: 0.119349
[216]	valid_0's binary_logloss: 0.118876
[217]	valid_0's binary_logloss: 0.118403
[218]	valid_0's binary_logloss: 0.117938
[219]	valid_0's binary_logloss: 0.11748
[220]	valid_0's binary_logloss: 0.117028
[221]	valid_0's binary_logloss: 0.116582
[222]	valid_0's binary_logloss: 0.11614
[223]	valid_0's binary_logloss: 0.115704
[224]	valid_0's binary_logloss: 0.115272
[225]	valid_0's binary_logloss: 0.114845
[226]	valid_0's binary_logloss: 0.114428
[227]	valid_0's bina

[401]	valid_0's binary_logloss: 0.0848874
[402]	valid_0's binary_logloss: 0.0848373
[403]	valid_0's binary_logloss: 0.084794
[404]	valid_0's binary_logloss: 0.0847466
[405]	valid_0's binary_logloss: 0.0846984
[406]	valid_0's binary_logloss: 0.0846551
[407]	valid_0's binary_logloss: 0.0846109
[408]	valid_0's binary_logloss: 0.08457
[409]	valid_0's binary_logloss: 0.0845279
[410]	valid_0's binary_logloss: 0.0844876
[411]	valid_0's binary_logloss: 0.0844455
[412]	valid_0's binary_logloss: 0.084404
[413]	valid_0's binary_logloss: 0.08436
[414]	valid_0's binary_logloss: 0.0843185
[415]	valid_0's binary_logloss: 0.0842779
[416]	valid_0's binary_logloss: 0.0842392
[417]	valid_0's binary_logloss: 0.0842
[418]	valid_0's binary_logloss: 0.0841622
[419]	valid_0's binary_logloss: 0.0841241
[420]	valid_0's binary_logloss: 0.0840875
[421]	valid_0's binary_logloss: 0.0840507
[422]	valid_0's binary_logloss: 0.0840152
[423]	valid_0's binary_logloss: 0.0839783
[424]	valid_0's binary_logloss: 0.083943
[4

[599]	valid_0's binary_logloss: 0.0811388
[600]	valid_0's binary_logloss: 0.0811308
[601]	valid_0's binary_logloss: 0.0811225
[602]	valid_0's binary_logloss: 0.081114
[603]	valid_0's binary_logloss: 0.0811098
[604]	valid_0's binary_logloss: 0.0811052
[605]	valid_0's binary_logloss: 0.0811016
[606]	valid_0's binary_logloss: 0.0810971
[607]	valid_0's binary_logloss: 0.0810906
[608]	valid_0's binary_logloss: 0.0810826
[609]	valid_0's binary_logloss: 0.0810775
[610]	valid_0's binary_logloss: 0.0810702
[611]	valid_0's binary_logloss: 0.0810635
[612]	valid_0's binary_logloss: 0.0810589
[613]	valid_0's binary_logloss: 0.0810547
[614]	valid_0's binary_logloss: 0.081047
[615]	valid_0's binary_logloss: 0.0810435
[616]	valid_0's binary_logloss: 0.081038
[617]	valid_0's binary_logloss: 0.0810316
[618]	valid_0's binary_logloss: 0.0810248
[619]	valid_0's binary_logloss: 0.0810188
[620]	valid_0's binary_logloss: 0.0810108
[621]	valid_0's binary_logloss: 0.0810053
[622]	valid_0's binary_logloss: 0.081

[800]	valid_0's binary_logloss: 0.0804847
[801]	valid_0's binary_logloss: 0.0804814
[802]	valid_0's binary_logloss: 0.0804803
[803]	valid_0's binary_logloss: 0.0804802
[804]	valid_0's binary_logloss: 0.0804776
[805]	valid_0's binary_logloss: 0.0804753
[806]	valid_0's binary_logloss: 0.0804743
[807]	valid_0's binary_logloss: 0.0804708
[808]	valid_0's binary_logloss: 0.080469
[809]	valid_0's binary_logloss: 0.0804677
[810]	valid_0's binary_logloss: 0.0804681
[811]	valid_0's binary_logloss: 0.0804667
[812]	valid_0's binary_logloss: 0.0804669
[813]	valid_0's binary_logloss: 0.0804661
[814]	valid_0's binary_logloss: 0.080463
[815]	valid_0's binary_logloss: 0.0804614
[816]	valid_0's binary_logloss: 0.0804594
[817]	valid_0's binary_logloss: 0.0804587
[818]	valid_0's binary_logloss: 0.0804579
[819]	valid_0's binary_logloss: 0.0804572
[820]	valid_0's binary_logloss: 0.0804562
[821]	valid_0's binary_logloss: 0.0804552
[822]	valid_0's binary_logloss: 0.0804553
[823]	valid_0's binary_logloss: 0.08

[996]	valid_0's binary_logloss: 0.0802941
[997]	valid_0's binary_logloss: 0.080295
[998]	valid_0's binary_logloss: 0.080294
[999]	valid_0's binary_logloss: 0.080295
[1000]	valid_0's binary_logloss: 0.0802945
[1001]	valid_0's binary_logloss: 0.080294
[1002]	valid_0's binary_logloss: 0.0802934
[1003]	valid_0's binary_logloss: 0.0802936
[1004]	valid_0's binary_logloss: 0.0802917
[1005]	valid_0's binary_logloss: 0.0802901
[1006]	valid_0's binary_logloss: 0.0802894
[1007]	valid_0's binary_logloss: 0.0802907
[1008]	valid_0's binary_logloss: 0.0802904
[1009]	valid_0's binary_logloss: 0.0802908
[1010]	valid_0's binary_logloss: 0.0802899
[1011]	valid_0's binary_logloss: 0.0802894
[1012]	valid_0's binary_logloss: 0.0802894
[1013]	valid_0's binary_logloss: 0.0802899
[1014]	valid_0's binary_logloss: 0.0802897
[1015]	valid_0's binary_logloss: 0.0802896
[1016]	valid_0's binary_logloss: 0.0802903
[1017]	valid_0's binary_logloss: 0.0802904
[1018]	valid_0's binary_logloss: 0.0802902
[1019]	valid_0's bi

[1188]	valid_0's binary_logloss: 0.0802478
[1189]	valid_0's binary_logloss: 0.0802481
[1190]	valid_0's binary_logloss: 0.0802483
[1191]	valid_0's binary_logloss: 0.080248
[1192]	valid_0's binary_logloss: 0.0802476
[1193]	valid_0's binary_logloss: 0.0802466
[1194]	valid_0's binary_logloss: 0.0802465
[1195]	valid_0's binary_logloss: 0.080247
[1196]	valid_0's binary_logloss: 0.0802474
[1197]	valid_0's binary_logloss: 0.0802487
[1198]	valid_0's binary_logloss: 0.0802502
[1199]	valid_0's binary_logloss: 0.0802494
[1200]	valid_0's binary_logloss: 0.0802488
[1201]	valid_0's binary_logloss: 0.0802486
[1202]	valid_0's binary_logloss: 0.0802488
[1203]	valid_0's binary_logloss: 0.0802478
[1204]	valid_0's binary_logloss: 0.0802477
[1205]	valid_0's binary_logloss: 0.0802468
[1206]	valid_0's binary_logloss: 0.080246
[1207]	valid_0's binary_logloss: 0.0802451
[1208]	valid_0's binary_logloss: 0.0802447
[1209]	valid_0's binary_logloss: 0.0802448
[1210]	valid_0's binary_logloss: 0.0802445
[1211]	valid_0

[1382]	valid_0's binary_logloss: 0.0802495
[1383]	valid_0's binary_logloss: 0.0802507
[1384]	valid_0's binary_logloss: 0.0802494
[1385]	valid_0's binary_logloss: 0.0802493
[1386]	valid_0's binary_logloss: 0.0802499
[1387]	valid_0's binary_logloss: 0.0802493
[1388]	valid_0's binary_logloss: 0.0802496
[1389]	valid_0's binary_logloss: 0.0802495
[1390]	valid_0's binary_logloss: 0.0802491
[1391]	valid_0's binary_logloss: 0.0802505
[1392]	valid_0's binary_logloss: 0.0802509
[1393]	valid_0's binary_logloss: 0.0802515
[1394]	valid_0's binary_logloss: 0.0802507
[1395]	valid_0's binary_logloss: 0.0802489
[1396]	valid_0's binary_logloss: 0.0802501
[1397]	valid_0's binary_logloss: 0.0802492
[1398]	valid_0's binary_logloss: 0.0802499
[1399]	valid_0's binary_logloss: 0.0802492
[1400]	valid_0's binary_logloss: 0.0802484
[1401]	valid_0's binary_logloss: 0.0802489
[1402]	valid_0's binary_logloss: 0.0802488
[1403]	valid_0's binary_logloss: 0.0802493
[1404]	valid_0's binary_logloss: 0.0802503
[1405]	vali

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=2, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [129]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0802392979662

Start predicting...
0.078150322105
0.0802392979662


In [91]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [92]:
y_sub_1.shape

(42888,)

In [93]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180419.txt',sep=" ",index=False)